In [1]:
from game import GAME
import torch
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
env = GAME(draw=True)  
observations = env.reset()

In [3]:
def rewards_function(information):
    colisoes = information['colisoes']
    bolas_caidas = information['bolas_caidas']
    perdeu = information['perdeu']
    ganhou = information['ganhou']
    joga_novamente = information['joga_novamente']
    bolas_jogador = information.get('bolas_jogador', [])
    bolas_adversario = information.get('bolas_adversario', [])
    winner = information.get('winner', None)
    penalizado = information.get('penalizado', False)

    rewards = 0

    if joga_novamente:
        rewards += 1

    if perdeu:
        rewards -= 1.5

    if ganhou:
        rewards += 1.5

    if penalizado:
        rewards -= 1

    return rewards

In [4]:
env.table.draw()

In [5]:
## jogador faz a jogada


env.iniciou_jogada = False
while not env.iniciou_jogada:
    env.table.draw()

obs, information, terminations, rewards = env.step((env.iniciou_jogada_angulo, 
                                                    env.inicou_jogada_intensidade), rewards_function=rewards_function)

print('state', obs)
print('termination', terminations)
print('rewards', rewards)

state (tensor([[0.0137, 0.5000, 1.0000, 0.0000],
        [0.7708, 0.2246, 1.0000, 0.0000],
        [0.6730, 0.4139, 1.0000, 0.0000],
        [0.8060, 0.4679, 1.0000, 0.0000],
        [0.7746, 0.8149, 1.0000, 0.0000],
        [0.8508, 0.5478, 1.0000, 0.0000],
        [0.4874, 0.4273, 1.0000, 0.0000],
        [0.7255, 0.6092, 1.0000, 0.0000],
        [0.7863, 0.5857, 1.0000, 0.0000],
        [0.7450, 0.8495, 1.0000, 0.0000],
        [0.7739, 0.5051, 1.0000, 0.0000],
        [0.8250, 0.6250, 1.0000, 0.0000],
        [0.8031, 0.5470, 1.0000, 0.0000],
        [0.7468, 0.0736, 1.0000, 0.0000],
        [0.7899, 0.1778, 1.0000, 0.0000]]), tensor([0.2000, 0.5000]))
termination False
rewards 0


In [6]:
# computador faz a jogada
print("Iniciando jogada", env.jogador_atual)



angulo = 10
intensidade = 0.9

env.iniciou_jogada = False
obs, information, terminations, rewards = env.step( (angulo, intensidade),  rewards_function=rewards_function)

print("Recompensa da jogada:", rewards)

Iniciando jogada True
Recompensa da jogada: 0


True

## Começando treinamento
                           

In [9]:
#import gymnasium as gym
#from stable_baselines3 import PPO 
#from stable_baselines3.common.env_checker import check_env
#from stable_baselines3.common.vec_env import DummyVecEnv
#

import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque


In [91]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerEncoder(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, num_layers, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        
        # make a linear
        self.embedding = nn.Linear(4, embed_dim)
        
        # Encoder layers
        self.encoder_layers = nn.ModuleList([
            nn.TransformerEncoderLayer( d_model=embed_dim, nhead=num_heads, dim_feedforward=ff_dim, dropout=dropout) for _ in range(num_layers)
        ])
        
        # Layer normalization
        self.layer_norm = nn.LayerNorm(embed_dim)
        
        
        self.mlp = nn.Sequential( 
                nn.Linear(embed_dim, ff_dim),
                nn.Tanh(),
                nn.Linear(ff_dim, 3)
        )
        
        self.head_position = nn.Tanh()
        self.head_intensity = nn.Sigmoid()

    def forward(self, x, bola_branca):
        
        # adiciona a dimensão da bolinha branca -> batch, (x,y,1,-1)
        b = bola_branca.shape[0]
        
        t_concat = torch.zeros(b,2, device=bola_branca.device)
        t_concat[:,0] = 1
        t_concat[:,1] = -1
        bola_branca = torch.concat( (bola_branca , t_concat), dim=-1).unsqueeze(1)
        
        x = torch.concat((bola_branca,x),dim=1)
        
                
        
        x = self.embedding(x)        
        
        for layer in self.encoder_layers:
            x = layer(x)
    
        x = self.layer_norm(x)
        
        # bola branca é nosso Value
        x = x[:,0,:]        
        x = self.mlp(x)
        
        
        # transforma o x em angulo e intensidade
        position = self.head_position(x[:,:2])
        intensity = self.head_intensity(x[:,2:])
        
        x_sin = position[:,0:1]
        x_cos = position[:,1:2]
        
        angles_deg = torch.rad2deg(torch.atan2(x_sin, x_cos))
        angles_deg = torch.where(angles_deg < 0, angles_deg + 360, angles_deg)
        
        
        # concatena o resultado
        return torch.concat((angles_deg,intensity),dim=-1)



# Parameters
embed_dim = 128         # Embedding dimension
num_heads = 8           # Number of attention heads
ff_dim = embed_dim * 2  # Feedforward dimension
num_layers = 2          # Number of encoder layers
dropout = 0.1           # Dropout rate


model = TransformerEncoder(
    embed_dim=embed_dim,
    num_heads=num_heads,
    ff_dim=ff_dim,
    num_layers=num_layers,
    dropout=dropout
)


In [95]:
state, bola_branca = (torch.tensor([
        [[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.8250, 0.3750, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],
        [[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.8250, 0.3750, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]
        ]),
                      
 torch.tensor([[0.2000, 0.5000],[0.2000, 0.5000]]))

model(state,bola_branca)

tensor([[278.0536,   0.6743],
        [290.7160,   0.6681]], grad_fn=<CatBackward0>)

In [101]:
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        
        # Convolução para os dados de posição (x, y)
        self.conv_position = nn.Conv1d(
            in_channels=1, out_channels=16, kernel_size=2, stride=2
        )
        
        # Convolução para os dados de estado (se é do adversário e se está na mesa)
        self.conv_state = nn.Conv1d(
            in_channels=1, out_channels=16, kernel_size=2, stride=2
        )
        
        # Fully connected layers
        self.fc1 = nn.Linear(482, 128)  # Ajustar com base nas características concatenadas
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_size)

    def forward(self, x):
        # Garantir que o tensor de entrada seja do tamanho correto
        assert x.numel() == 62, f"Expected input of size 62, got {x.numel()} elements."

        # Separar dados de posição e estado
        position_data = x[0:60].reshape(-1, 4)[:, 0:2].reshape(1, 1, -1)  # Extrai (x, y) pares consecutivos
        state_data = x[0:60].reshape(-1, 4)[:, 2:4].reshape(1, 1, -1)  # Extrai (adversário, na mesa)
        white_ball_data = x[60:62]  # Dados da bola branca (x, y)

        # Aplicar convoluções
        position_features = torch.relu(self.conv_position(position_data))
        state_features = torch.relu(self.conv_state(state_data))

        # Flatten convoluções
        position_features = position_features.view(-1)  # Achatar
        state_features = state_features.view(-1)        # Achatar
        
        
        #print(f'Position features shape: {position_features.shape}')
        #print(f'State features shape: {state_features.shape}')

        # Concatenar características das convoluções e dados da bola branca
        combined_features = torch.cat([position_features, state_features, white_ball_data], dim=0)
        
        #print(f'Combined features shape: {combined_features.shape}')

        # Passar pelas fully connected layers
        x = torch.relu(self.fc1(combined_features))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
import utils.config as cfg

## iniciar o ambiente com a configuração do jogo com apenas uma bola na mesa. 
# - A bola branca na posição (0, 0)
# - se não bater na bola, acabou a jogada

cfg.raio_buraco = 13
env.reset()
env.table.bolas = env.table.bolas[1:2] + [env.table.bola_branca]
numero_bola = env.table.bolas[0].numero
env.numero_bolas = [[ numero_bola ], [ numero_bola ]]

#env.table.draw()



In [34]:
state_bolas, state_bola_branca = env.get_observations()
# shuffle the balls
state_bolas = state_bolas[torch.randperm(state_bolas.size()[0])]
state_bolas , state_bola_branca

(tensor([[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.8250, 0.3750, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]),
 tensor([0.2000, 0.5000]))

In [102]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.99  # Fator de desconto
        self.epsilon = 1.0  # Taxa de exploração
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.batch_size = 64

        self.model = DQN(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.criterion = nn.MSELoss()

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            # Exploração: escolher valores aleatórios dentro do intervalo permitido
            angle = np.random.uniform(0, 360)  # Ângulo contínuo entre 0° e 360°
            intensity = np.random.uniform(0, 1)  # Intensidade contínua entre 0 e 1
            #print(f"Explorando: Ângulo={angle:.2f}, Intensidade={intensity:.2f}")
            return np.array([angle, intensity])
            
        else:
            # Exploração: usar a política aprendida
            state = torch.FloatTensor(state)
            q_values = self.model(state)
            action = q_values.detach().numpy()  # Retorna o vetor [ângulo, intensidade]
            #print(f"Explotando: Ângulo={action[0]:.2f}, Intensidade={action[1]:.2f}")
            return action



    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        minibatch = random.sample(self.memory, self.batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            next_state = torch.FloatTensor(next_state).view(-1)  # Garantir formato correto
            
            if not done:
                target += self.gamma * torch.max(self.model(next_state).detach())
            
            state = torch.FloatTensor(state).view(-1)  # Garantir formato correto
            target_f = self.model(state)

            # Calculando a perda diretamente
            loss = self.criterion(target_f, torch.FloatTensor(action))
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay




In [103]:
env = CustomEnvWrapper(GAME(draw=False))

# Inicializar o tamanho do estado e das ações
state_size = 62  # Tamanho do estado processado
action_size = 2  # Ângulo e Intensidade

# Inicializar o agente
agent = DQNAgent(state_size=state_size, action_size=action_size)

In [114]:
env = CustomEnvWrapper(GAME(draw=False))

In [119]:
# Definir o número de episódios para o treinamento
episodes = 500

window_size = 10  # Tamanho da janela para a média móvel
reward_window = deque(maxlen=window_size) 

for e in range(episodes):
    state, _ = env.reset()  # Obter o estado inicial como array NumPy
    total_reward = 0
    done = False

    c = 0  # Contador para print debug

    while not done:
        c += 1
        # Escolher uma ação contínua
        action = agent.act(state)  # Retorna [angle, intensity]
        angle, intensity = float(action[0]), float(action[1])

        # Realizar a ação no ambiente
        obs, reward, done, _ = env.step((angle, intensity))
        
        # Usar diretamente o estado retornado
        next_state = obs

        # Memorizar a transição
        agent.remember(state, action, reward, next_state, done)

        # Atualizar o estado atual
        state = next_state
        total_reward += reward

        # Debug a cada 10 iterações
        #if c % 10 == 0:
            #print(f'Action: Angle={angle}, Intensity={intensity}, Reward={reward}, Total Reward={total_reward}')
            
            
        reward_window.append(total_reward)

        if done:
            #print()
            moving_avg = sum(reward_window) / len(reward_window)

            print(f"Episode {e+1}/{episodes} - Total Reward: {total_reward} - Moving Average: {moving_avg:.2f}")
            #print()
            break

    # Treinar o agente
    agent.replay()

Episode 1/500 - Total Reward: -105 - Moving Average: 30.00
Episode 2/500 - Total Reward: 9970 - Moving Average: 10105.00
Episode 3/500 - Total Reward: 550 - Moving Average: 685.00
Episode 4/500 - Total Reward: 3565 - Moving Average: 3700.00
Episode 5/500 - Total Reward: 3235 - Moving Average: 3370.00
Episode 6/500 - Total Reward: -100 - Moving Average: 31.50
Episode 7/500 - Total Reward: 670 - Moving Average: 805.00
Episode 8/500 - Total Reward: -30 - Moving Average: 105.00
Episode 9/500 - Total Reward: 3160 - Moving Average: 3263.50
Episode 10/500 - Total Reward: 250 - Moving Average: 227.50
Episode 11/500 - Total Reward: 1150 - Moving Average: 1285.00
Episode 12/500 - Total Reward: 3225 - Moving Average: 3360.00
Episode 13/500 - Total Reward: 55 - Moving Average: 186.50
Episode 14/500 - Total Reward: 5980 - Moving Average: 5957.50
Episode 15/500 - Total Reward: 9370 - Moving Average: 9347.50
Episode 16/500 - Total Reward: 14950 - Moving Average: 14927.50
Episode 17/500 - Total Reward

KeyboardInterrupt: 

In [69]:
lista = [1, 2, 3, 4, 5, 6]

lista[2:-1: 2]

[3, 5]

In [120]:
import torch

save_path = "dqn_agent_model6.pth"

# Salvar o modelo
torch.save({
    'model_state_dict': agent.model.state_dict(),
    'optimizer_state_dict': agent.optimizer.state_dict(),
    'epsilon': agent.epsilon,  # Salvar o epsilon atual
}, save_path)

print(f"Modelo salvo em {save_path}")


Modelo salvo em dqn_agent_model6.pth


In [12]:
model_path = "dqn_agent_model.pth"

# Inicializar o agente
agent = DQNAgent(state_size=62, action_size=2)

# Carregar os pesos do modelo e do otimizador
checkpoint = torch.load(model_path)
agent.model.load_state_dict(checkpoint['model_state_dict'])
agent.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
agent.epsilon = checkpoint['epsilon']  # Restaurar o epsilon atual

print("Modelo carregado com sucesso!")

Modelo carregado com sucesso!


/var/folders/ys/qvr7rwln3yjg1zwjrrspn9gc0000gn/T/ipykernel_62796/3586500898.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path)


DQN(
  (fc1): Linear(in_features=62, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)

In [16]:
env = GAME(draw=False)  
observations = env.reset()

In [ ]:
print("Iniciando jogada", env.jogador_atual)

done = False  # Variável para controlar o término do jogo
total_reward = 0  # Variável para acompanhar a recompensa total

while not done:
    while not env.iniciou_jogada:
        env.table.draw()  # Renderizar o estado atual do jogo (se draw=True)

    # O agente escolhe uma ação
    state = obs  # Assumimos que 'obs' é o estado retornado do ambiente
    action = agent.act(state)  # Agente escolhe o ângulo e a intensidade
    angle, intensity = float(action[0]), float(action[1])

    # Realizar a jogada no ambiente
    obs, information, terminations, rewards = env.step(
        (angle, intensity), 
        rewards_function=rewards_function
    )

    env.iniciou_jogada = False  # Resetar o estado da jogada
    total_reward += rewards  # Atualizar a recompensa total
    print(f"Ação realizada: Ângulo={angle}, Intensidade={intensity}")
    print(f"Recompensa da jogada: {rewards}")

    # Verificar se o jogo terminou
    done = terminations

print(f"Fim do jogo! Recompensa total: {total_reward}")


Iniciando jogada 0
Ação realizada: Ângulo=114.47925567626953, Intensidade=0.5772977471351624
Recompensa da jogada: 0


ValueError: only one element tensors can be converted to Python scalars